In [1]:
import tensorflow as tf

In [3]:

filename_queue = tf.train.string_input_producer(["data/train.csv"])

reader = tf.TextLineReader(skip_header_lines=1)
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[1] for i in range(541)]
# print(record_defaults)

cols = tf.decode_csv(value, record_defaults=record_defaults)
print(cols[0])

labels = tf.one_hot(cols[1], 2)
features = tf.stack(cols[2:])
print(labels)
print(features)

batch_size = 100
min_after_dequeue = 10000
capacity = min_after_dequeue + 3 * batch_size
    
example_batch, label_batch = tf.train.shuffle_batch(
        [features, labels], batch_size=batch_size, capacity=capacity,
        min_after_dequeue=min_after_dequeue)
print(example_batch)
print(label_batch)

Tensor("shuffle_batch:0", shape=(100, 539), dtype=int32)
Tensor("shuffle_batch:1", shape=(100, 2), dtype=float32)


In [9]:

#define place holder
x = tf.placeholder(tf.float32, [None, 539])
y = tf.placeholder(tf.float32, [None, 2])

#create a 1-level neural network
W = tf.Variable(tf.truncated_normal([539, 2], stddev=0.1))
# b = tf.Variable(tf.zeros([2]))
b = tf.Variable(tf.zeros([2])+0.1)

wx_b = tf.matmul(x, W)+b

#激活函数，实现非线性化
# prediction = tf.nn.relu(wx_b)
# prediction = tf.nn.tanh(wx_b)
# prediction = tf.nn.sigmoid(wx_b)
prediction = tf.nn.softmax(wx_b)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
# loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=prediction))

#train
train = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

init = tf.global_variables_initializer()

#比较预测值与实际值
correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax 返回沿着某个维度最大值的位置
`
#准确率
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


with tf.Session() as sess:
    sess.run(init)
    
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for i in range(1):
        # Retrieve a single instance:
        batch_xs, batch_ys = sess.run([example_batch, label_batch])
        print(batch_xs.shape)
        print(batch_xs)
    
        sess.run(train, feed_dict={x: batch_xs, y: batch_ys})


    coord.request_stop()
    coord.join(threads)

(100, 539)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


In [ ]:
x_data = np.linspace(-0.5, 0.5, 200)[:, np.newaxis]
noise = np.random.normal(0, 0.02, x_data.shape)
y_data = np.square(x_data) + noise

print(x_data)